# Quantify and Correct Template for Contamination

Contact author: Melissa Graham

Date last verified to run: Fri Dec 23, 2022

RSP environment version: Weekly 2022_40


**STATUS** Needs to be redone. Using the `DiaObject` minimum detected flux to evaluate the fraction of SNIa that experience template contamination seems to be a great overestimate when compared with the fraction derived from using the `ForcedSourceOnDiaObjects`. So start it all over but only use `ForcedSourceOnDiaObjects`.


## 1. Introduction

For DP0.2, the template images were constructed using the 30% of the images with the best seeing over the 6-year DC2 simulation.

This results in some flux in the template images ("template contamination") for some supernovae which is then subtracted during difference image analysis, leading to negative or reduced difference-image fluxes.

Section 3 demonstrates that ~45% of SNIa lightcurves are affected by template contamination.

Section 4 demonstrates a method for correcting for template contamination using the `ForcedSourceOnDiaObjects` catalog, which has an uncertainty of 10-20%.

### 1.1. Import packages

In [ ]:
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import astropy.units as u

import pandas
pandas.set_option('display.max_rows', 1000)

from lsst.rsp import get_tap_service, retrieve_query

### 1.2. Set global parameters and functions

In [ ]:
service = get_tap_service()

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u' : '#56b4e9', 'g' : '#008060', 'r' : '#ff4000',
                      'i' : '#850000', 'z' : '#6600cc', 'y' : '#000000'}
plot_filter_symbols = {'u' : 'o', 'g' : '^', 'r' : 'v', 
                       'i' : 's', 'z' : '*', 'y' : 'p'}

## 2. Establish a sample of SNIa

### 2.1. Get a large sample of z<0.5 true SNIa

Use a radius = 4 degree cone search near the center of DC2.

This should return around 10416 true SNIa and take < 2 minutes.

In [ ]:
%%time
query = "SELECT mt.id_truth_type, ts.ra, ts.dec, ts.truth_type "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 4)) = 1 "\
        "AND ts.truth_type = 3 AND ts.redshift < 0.5"
results = service.search(query)
del query

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
TrueSNIa = results.to_table().to_pandas()
del results
print('len(TrueSNIa) = ', len(TrueSNIa))

Option to display table contents.

In [ ]:
# TrueSNIa

### 2.2. Get a large sample of `DiaObjects`

Retrieve DiaObjects from the same region which were detected with a maximum flux of at least 1000 nJy in the r- and i-bands, and at least 5 detections in r- and i-bands.

SNIa that have a redshift < 0.5 and were detected near peak brightness would meet this criteria.

This should return 116953 `DiaObjects` and take < 1 minute.

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
%%time
query = "SELECT diaObjectId, ra, decl "\
        "FROM dp02_dc2_catalogs.DiaObject "\
        "WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 4)) = 1 "\
        "AND rPSFluxMax > 1000 AND iPSFluxMax > 1000 "\
        "AND rPSFluxNdata > 5 AND iPSFluxNdata > 5"
results = service.search(query)
del query

Convert results to a pandas table with `to_table().to_pandas()`.

In [ ]:
DiaObjs = results.to_table().to_pandas()
del results
print('len(DiaObjs) = ', len(DiaObjs))

Option to display table contents.

In [ ]:
# DiaObjs

### 2.3. Match `TrueSNIa` to `DiaObjs`

First, use `astropy` to create arrays of sky coordinates (RA, Dec) using `SkyCoord`.

In [ ]:
TrueSNIa_skycoord = SkyCoord(ra=TrueSNIa.loc[:, 'ra'].values*u.degree, 
                             dec=TrueSNIa.loc[:, 'dec'].values*u.degree)
DiaObjs_skycoord = SkyCoord(ra=DiaObjs.loc[:, 'ra'].values*u.degree, 
                            dec=DiaObjs.loc[:, 'decl'].values*u.degree)

Use `match_coordinates_sky` to find the nearest `DiaObjs` table entry for each `TrueSNIa` table entry.

`idx` = the index in `DiaObjs` to the nearest match of each `TrueSNIa`

`d2d` = the two-dimenstional distance to the nearest match, in arcsec

`d3d` = a three-dimensional distance is returned, but meaningless for our use-case

In [ ]:
idx, d2d, d3d = match_coordinates_sky(TrueSNIa_skycoord, DiaObjs_skycoord)

In the figure below, the distribution of 2d distances to the nearest `DiaObjs` for each `TrueSNIa` shows that applying a maximum of 1 arcsecond as a "real" match would be appropriate.

In [ ]:
fig = plt.figure(figsize=(4, 2))
plt.hist(np.log10(d2d.arcsec), bins=50)
plt.xlabel('log distance to nearest match [arcsec]')
plt.show()

Print the number of `TrueSNIa` that are matched within 1 arcsecond of a `DiaObjs`.

This will show how many `DiaObjs` there are with which to explore template contamination.

In [ ]:
tx = np.where(d2d.arcsec < 1.0)[0]
print(len(tx))
del tx

Add columns to the `TrueSNIa` table to store the match information for good matches.

Use negative values as the placeholder.

In [ ]:
TrueSNIa['DiaObjs_idx'] = np.zeros(len(TrueSNIa), dtype='int') - 1
TrueSNIa['DiaObjs_d2d'] = np.zeros(len(TrueSNIa), dtype='float') - 1.0

Fill in the database with match information for the good matches only.

In [ ]:
d2d_arcsec = d2d.arcsec
tx = np.where(d2d.arcsec < 1.0)[0]
for x in tx:
    TrueSNIa.loc[x, 'DiaObjs_idx'] = idx[x]
    TrueSNIa.loc[x, 'DiaObjs_d2d'] = d2d_arcsec[x]
del tx, d2d_arcsec
del idx, d2d, d3d

Option to see how the two columns have been added to the table, with negative values used as placeholders for no match information.

In [ ]:
# TrueSNIa

## 3. Estimate fraction of SNIa affected by template contamination

Without template contamination, supernovae would never be detected with a negative flux value in a difference image.

In other words, the minimum `DiaSource` difference-image fluxes for `DiaObjects` matched with true SNIa _should_ always be positive.

The fraction which are negative indicates approximately how many true SNIa `DiaObjects` are impacted by template contamination.


### 3.1. Retrieve minimum detected difference-image fluxes for true SNIa

First create a string, formatted as a tuple, that contains a list of all the `diaObjectId` values for the elements of `DiaObjs` matched to `TrueSNIa`.

In [ ]:
tuple_string_diaObjectId = '('
tx = np.where(TrueSNIa.loc[:, 'DiaObjs_idx'] >= 0)[0]
for i, x in enumerate(tx):
    tuple_string_diaObjectId += str(DiaObjs.loc[TrueSNIa.loc[x, 'DiaObjs_idx'], 'diaObjectId'])
    if i < len(tx)-1:
        tuple_string_diaObjectId += ','
    else:
        tuple_string_diaObjectId += ')'
del tx

Option to view the string-formatted tuple.

In [ ]:
# print(tuple_string_diaObjectId)

After this, we don't use the DiaObjs or TrueSNIa dataframes again.

In [ ]:
del DiaObjs, TrueSNIa

Retrieve the minimum difference-image flux detections from the `DiaObject` catalog.

In [ ]:
%%time
query = "SELECT diaObjectId, uPSFluxMin, gPSFluxMin, rPSFluxMin, "\
        "iPSFluxMin, zPSFluxMin, yPSFluxMin "\
        "FROM dp02_dc2_catalogs.DiaObject "\
        "WHERE diaObjectId IN "+tuple_string_diaObjectId
results = service.search(query)
del query

In [ ]:
DiaObj_PSFluxMin = results.to_table().to_pandas()
del results

### 3.2. Plot distribution of minimum detected difference-image fluxes for true SNIa

Below, plot the distribution of minimum detected fluxes (i.e., from `DiaSources`) by filter (colored histograms) for `DiaObjects` matched to SNIa.

Notice that the x-axis limits for the 'y' band are broader.

Grey lines indicate the _approximate_ 5-sigma detection limit in the processed visit images (PVI; direct images).

`depths_PVI_mags` = the approximate 5-sigma limiting magnitudes for a single direct image, in magnitudes

`SNR5_PVI_nJy` = same as `depths_PVI_mags`, but in flux

In [ ]:
depths_PVI_mags = np.asarray([23.9,25.0,24.7,24.0,23.3,22.1], dtype='float')
SNR5_PVI_nJy = 10.0**((depths_PVI_mags - 31.4) / -2.5)

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axvline(-1.0 * SNR5_PVI_nJy[f], alpha=0.5, color='grey')
        ax[i,j].axvline(SNR5_PVI_nJy[f], alpha=0.5, color='grey', label='SNR=5')

        tx = np.where((DiaObj_PSFluxMin[filt+'PSFluxMin'] > -20000)
                      & (DiaObj_PSFluxMin[filt+'PSFluxMin'] < 20000))[0]
        ax[i,j].hist(DiaObj_PSFluxMin.loc[tx, filt+'PSFluxMin'], bins=100, histtype='step',
                     lw=2, color=plot_filter_colors[filt], label=filt)
        del tx

        ax[i,j].legend(loc='best')
        if filt == 'y':
            ax[i,j].set_xlim([-20000, 20000])
        else:
            ax[i,j].set_xlim([-5000, 5000])
        f += 1

ax[2,0].set_xlabel('DiaSource PSFluxMin')
ax[2,1].set_xlabel('DiaSource PSFluxMin')
ax[0,0].set_ylabel('# DiaObjects')
ax[1,0].set_ylabel('# DiaObjects')
ax[2,0].set_ylabel('# DiaObjects')
plt.show()

Above, it is a bit odd that there are detections within the vertical lines, but this is probably (at least in part) due to the fact that _estimated_ PVI depths are used.

### 3.3. Calculate fraction of true SNIa with "significantly negative" minimum detected difference-image fluxes

Below, calculate the fraction of `DiaObjects` matched to SNIa that have a `PSFluxMin` that was detected to be significantly negative (i.e., with SNR > 5), for each filter.

This is approximately the number of true SNIa affected by template contamination, per filter.

In [ ]:
frac_filt = np.zeros(6, dtype='float')
for f, filt in enumerate(plot_filter_labels):
    fx = np.where(np.isfinite(DiaObj_PSFluxMin[filt+'PSFluxMin']))[0]
    tx = np.where(DiaObj_PSFluxMin.loc[fx, filt+'PSFluxMin'] < -1.0 * SNR5_PVI_nJy[f])[0]
    frac_filt[f] = np.round(float(len(tx)) / float(len(fx)), 3)
    print(filt, frac_filt[f])
del frac_filt

Calculate the fraction of `DiaObjects` matched to SNIa that have a `PSFluxMin` that was detected to be significantly negative _in any filter_. 

In [ ]:
flag = np.zeros(len(DiaObj_PSFluxMin), dtype='int')
for i in range(len(DiaObj_PSFluxMin)):
    for f, filt in enumerate(plot_filter_labels):
        if (np.isfinite(DiaObj_PSFluxMin.loc[i, filt+'PSFluxMin'])) \
           & (DiaObj_PSFluxMin.loc[i, filt+'PSFluxMin'] < -1.0 * SNR5_PVI_nJy[f]):
            flag[i] = 1

tx = np.where(flag == 1)[0]
print(np.round(float(len(tx)) / float(len(DiaObj_PSFluxMin)), 3))
del flag, tx

**Thus, the fraction of true SNIa affected by template contamination is about 75%.**

**That seems way too high?**

This does not include true SNIa for which there was a "small" amount of flux in the template, where "small" means an amount of flux less than the 5-sigma limiting flux for a single direct image in that filter. 

In [ ]:
del DiaObj_PSFluxMin

## 4. Generate a correction for template contamination

Use the forced photometry in the `ForcedSourceOnDiaObjects` table to generate and then apply a correction for template contamination.

### 4.1. Retrieve difference-image forced photometry

The PSF-fit fluxes are available in `ForcedSourceOnDiaObjects`, but to get the visit dates requires a join with the `CcdVisit` table.

Retrieve data from `ForcedSourceOnDiaObjects` for all forced sources associated with a `DiaObject` which was matched to a true SNIa.

In [ ]:
%%time
query = "SELECT fsodo.diaObjectId, fsodo.ccdVisitId, "\
        "fsodo.psfDiffFlux, fsodo.psfDiffFluxErr, fsodo.psfDiffFlux_flag, "\
        "ccdv.ccdVisitId, ccdv.band, ccdv.expMidptMJD "\
        "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject AS fsodo "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS ccdv ON ccdv.ccdVisitId = fsodo.ccdVisitId "\
        "WHERE fsodo.diaObjectId IN "+tuple_string_diaObjectId
results = service.search(query)
del query

Convert results to a pandas table.

In [ ]:
FSrc = results.to_table().to_pandas()
del results

Option to view the table of data retrieved.

In [ ]:
# FSrc

### 4.2. Identify dates of peak brightess 

First, make an array of the unique `diaObjectid` for the `DiaObjects` matched to true SNIa, for which forced photometry was retrieved from the `ForcedSourceOnDiaObject` table.

In [ ]:
SNIa_diaObjectId = np.unique(FSrc.loc[:, 'diaObjectId'])

Next, make an array to hold the date of the brightest detection in MJD.

In [ ]:
SNIa_peak_date = np.zeros(len(SNIa_diaObjectId), dtype='float')

Determine the peak date from the difference-image fluxes.

In [ ]:
%%time
for i, id_val in enumerate(SNIa_diaObjectId):
    sx = np.where(FSrc['diaObjectId'] == id_val)[0]
    mx = np.argmax(FSrc.loc[sx, 'psfDiffFlux'])
    SNIa_peak_date[i] = FSrc.loc[sx[mx], 'expMidptMJD']
    del sx, mx

### 4.3. Calculate flux correction factor and uncertainty

First, make arrays to hold the mean and standard deviation (`std`) of the difference-image forced flux in the "off-peak" visits (i.e., >300 days before, or >500 days after, the peak date), along with the number of visits (`N`) that contributed, per filter.

In [ ]:
SNIa_offpeak_min = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_max = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_mean = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_std = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_N = np.zeros((len(SNIa_diaObjectId), 6), dtype='int')

Fill the arrays.

Only use unflagged values of `psfDiffFlux`. 

This takes about 30 seconds to compute.

In [ ]:
%%time
for i, id_val in enumerate(SNIa_diaObjectId):
    sx = np.where(FSrc['diaObjectId'] == id_val)[0]
    if len(sx) > 0:
        for f, filt in enumerate(plot_filter_labels):
            fx = np.where(FSrc.loc[sx, 'band'] == filt)[0]
            if len(fx) > 0:
                dx = np.where((FSrc.loc[sx[fx], 'psfDiffFlux_flag'] == 0)
                              & ((FSrc.loc[sx[fx], 'expMidptMJD'] <= SNIa_peak_date[i]-300) 
                                 |(FSrc.loc[sx[fx], 'expMidptMJD'] >= SNIa_peak_date[i]+500)))[0]
                SNIa_offpeak_N[i, f] = len(dx)
                if len(dx) > 0:
                    SNIa_offpeak_min[i, f] = np.min(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_max[i, f] = np.max(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_mean[i, f] = np.mean(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                    SNIa_offpeak_std[i, f] = np.std(FSrc.loc[sx[fx[dx]], 'psfDiffFlux'])
                del dx
            del fx
    del sx

Plot the distribution of minimum (dotted), maximum (dashed), and mean (solid) difference-image forced photometry fluxes for the `DiaObjects` that are matched to a true SNIa.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axvline(-1.0 * SNR5_PVI_nJy[f], alpha=0.5, color='grey')
        ax[i,j].axvline(SNR5_PVI_nJy[f], alpha=0.5, color='grey', label='SNR>5')

        tx = np.where((SNIa_offpeak_N[:, f] >= 3)
                      & (SNIa_offpeak_min[:, f] > -20000)
                      & (SNIa_offpeak_min[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_min[tx, f], bins=150, histtype='step', log=True,
                     lw=1, ls='dotted', color=plot_filter_colors[filt], label='min')
        del tx
        
        tx = np.where((SNIa_offpeak_N[:, f] >= 3)
                      & (SNIa_offpeak_max[:, f] > -20000)
                      & (SNIa_offpeak_max[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_max[tx, f], bins=150, histtype='step', log=True,
                     lw=1, ls='dashed', color=plot_filter_colors[filt], label='max')
        del tx

        tx = np.where((SNIa_offpeak_N[:, f] >= 3)
                      & (SNIa_offpeak_mean[:, f] > -20000)
                      & (SNIa_offpeak_mean[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_mean[tx, f], bins=150, histtype='step', log=True,
                     lw=2, ls='solid', color=plot_filter_colors[filt], label='mean')
        del tx

        if filt == 'y':
            ax[i,j].set_xlim([-20000, 20000])
        else:
            ax[i,j].set_xlim([-5000, 5000])

        ax[i,j].legend(loc='best')
        f += 1

ax[2,0].set_xlabel('Off-peak Forced Flux')
ax[2,1].set_xlabel('Off-peak Forced Flux')
ax[0,0].set_ylabel('# DiaObjects')
ax[1,0].set_ylabel('# DiaObjects')
ax[2,0].set_ylabel('# DiaObjects')
plt.show()

It's very strange that in the above plots, the distributions of the minimum fluxes (dotted) are not similar to the left component of the distributions plotted using the `PSFluxMin`, above.

If we were using only the `ForcedSourceOnDiaObject` table, what would be the fraction of true SNIa that have an average off-peak forced flux that is "significantly negative"?

In [ ]:
frac_filt = np.zeros(6, dtype='float')
for f, filt in enumerate(plot_filter_labels):
    fx = np.where(np.isfinite(SNIa_offpeak_mean[:, f]))[0]
    tx = np.where(SNIa_offpeak_mean[fx, f] < -1.0 * SNR5_PVI_nJy[f])[0]
    frac_filt[f] = np.round(float(len(tx)) / float(len(fx)), 3)
    print(filt, frac_filt[f])
    del fx, tx
del frac_filt

Much lower than before!

Perhaps there is something going on here, could the minimum fluxes from `DiaSource` really be so different?

### 4.3. Calculate flux correction factor and uncertainty with `DiaSource`

In [ ]:
query = "SELECT ds.diaObjectId, ds.ccdVisitId, "\
        "ds.psFlux, ds.psFluxErr, ds.psfFlux_flag, "\
        "ccdv.ccdVisitId, ccdv.band, ccdv.expMidptMJD "\
        "FROM dp02_dc2_catalogs.DiaSource AS ds "\
        "JOIN dp02_dc2_catalogs.CcdVisit AS ccdv ON ccdv.ccdVisitId = ds.ccdVisitId "\
        "WHERE ds.diaObjectId IN "+tuple_string_diaObjectId
results = service.search(query)
del query

In [ ]:
DSrc = results.to_table().to_pandas()
del results

In [ ]:
SNIa_offpeak_ds_min = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_ds_max = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_ds_mean = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_ds_std = np.zeros((len(SNIa_diaObjectId), 6), dtype='float')
SNIa_offpeak_ds_N = np.zeros((len(SNIa_diaObjectId), 6), dtype='int')

In [ ]:
%%time
for i, id_val in enumerate(SNIa_diaObjectId):
    sx = np.where(DSrc['diaObjectId'] == id_val)[0]
    if len(sx) > 0:
        for f, filt in enumerate(plot_filter_labels):
            fx = np.where(DSrc.loc[sx, 'band'] == filt)[0]
            if len(fx) > 0:
                dx = np.where(((DSrc.loc[sx[fx], 'expMidptMJD'] <= SNIa_peak_date[i]-300) 
                               | (DSrc.loc[sx[fx], 'expMidptMJD'] >= SNIa_peak_date[i]+500)))[0]
                SNIa_offpeak_ds_N[i, f] = len(dx)
                if len(dx) > 0:
                    SNIa_offpeak_ds_min[i, f] = np.min(DSrc.loc[sx[fx[dx]], 'psFlux'])
                    SNIa_offpeak_ds_max[i, f] = np.max(DSrc.loc[sx[fx[dx]], 'psFlux'])
                    SNIa_offpeak_ds_mean[i, f] = np.mean(DSrc.loc[sx[fx[dx]], 'psFlux'])
                    SNIa_offpeak_ds_std[i, f] = np.std(DSrc.loc[sx[fx[dx]], 'psFlux'])
                del dx
            del fx
    del sx

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        
        ax[i,j].axvline(-1.0 * SNR5_PVI_nJy[f], alpha=0.5, color='grey')
        ax[i,j].axvline(SNR5_PVI_nJy[f], alpha=0.5, color='grey', label='SNR>5')

        tx = np.where((SNIa_offpeak_ds_N[:, f] >= 3)
                      & (SNIa_offpeak_ds_min[:, f] > -20000)
                      & (SNIa_offpeak_ds_min[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_ds_min[tx, f], bins=150, histtype='step',
                     lw=1, ls='dotted', color=plot_filter_colors[filt], label='min')
        del tx
        
        tx = np.where((SNIa_offpeak_ds_N[:, f] >= 3)
                      & (SNIa_offpeak_ds_max[:, f] > -20000)
                      & (SNIa_offpeak_ds_max[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_ds_max[tx, f], bins=150, histtype='step',
                     lw=1, ls='dashed', color=plot_filter_colors[filt], label='max')
        del tx

        tx = np.where((SNIa_offpeak_ds_N[:, f] >= 3)
                      & (SNIa_offpeak_ds_mean[:, f] > -20000)
                      & (SNIa_offpeak_ds_mean[:, f] < 20000))[0]
        ax[i,j].hist(SNIa_offpeak_ds_mean[tx, f], bins=100, histtype='step',
                     lw=2, ls='solid', color=plot_filter_colors[filt], label='mean')
        del tx

        if filt == 'y':
            ax[i,j].set_xlim([-20000, 20000])
        else:
            ax[i,j].set_xlim([-5000, 5000])

        ax[i,j].legend(loc='best')
        f += 1

ax[2,0].set_xlabel('Off-peak DiaSource Flux')
ax[2,1].set_xlabel('Off-peak DiaSource Flux')
ax[0,0].set_ylabel('# DiaObjects')
ax[1,0].set_ylabel('# DiaObjects')
ax[2,0].set_ylabel('# DiaObjects')
plt.show()

In [ ]:
frac_filt = np.zeros(6, dtype='float')
for f, filt in enumerate(plot_filter_labels):
    fx = np.where(np.isfinite(SNIa_offpeak_ds_mean[:, f]))[0]
    tx = np.where(SNIa_offpeak_ds_mean[fx, f] < -1.0 * SNR5_PVI_nJy[f])[0]
    frac_filt[f] = np.round(float(len(tx)) / float(len(fx)), 3)
    print(filt, frac_filt[f])
    del fx, tx
del frac_filt

OK this is showing that when `DiaObjects` are detected in off-peak visits, they are detected with a negative flux.

BUT, it is only the `DiaObjects` with template contamination that _would_ be detected off-peak, and in that case, they would be detected with a negative flux.

Whereas when we use forced photometry, it is for everything.

How many SNIa are actually _detected_ during the off-peak visits?

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

f = 0
for i in range(3):
    for j in range(2):
        filt = plot_filter_labels[f]
        tx1 = np.where(SNIa_offpeak_ds_N[:, f] >= 20)[0]
        tx = np.where(SNIa_offpeak_ds_N[:, f] < 20)[0]
        ax[i,j].hist(SNIa_offpeak_ds_N[tx, f], bins=21, range=(-0.5,20.5), histtype='step',
                     log=True, lw=1, ls='solid', color=plot_filter_colors[filt],
                     label=filt+' (+'+str(int(len(tx1)))+' with N>20)')
        del tx, tx1

        ax[i,j].legend(loc='best')
        f += 1

ax[2,0].set_xlabel('# Off-peak Detections')
ax[2,1].set_xlabel('# Off-peak Detections')
ax[0,0].set_ylabel('# DiaObjects')
ax[1,0].set_ylabel('# DiaObjects')
ax[2,0].set_ylabel('# DiaObjects')
plt.show()

### 4.5. For off-peak detections, plot detected vs. forced flux

In [ ]:
%%time
t1 = time.time()
fig, ax = plt.subplots(3, 2, figsize=(14,6), sharex=False)

for sn in range(len(SNIa_diaObjectId)):
    id_val = SNIa_diaObjectId[sn]
    if sn == 100:
        t2 = time.time()
        print('100 SN in, ', t2-t1)
    if sn == 500:
        t2 = time.time()
        print('500 SN in, ', t2-t1)
        
    ds_sx = np.where(DSrc.loc[:, 'diaObjectId'] == id_val)[0]
    fs_sx = np.where(FSrc.loc[:, 'diaObjectId'] == id_val)[0]
    
    f = 0
    if (len(ds_sx) > 1) & (len(fs_sx) > 1):
        for i in range(3):
            for j in range(2):
                filt = plot_filter_labels[f]
                ds_fx = np.where(DSrc.loc[ds_sx, 'band'] == filt)[0]
                fs_fx = np.where(FSrc.loc[fs_sx, 'band'] == filt)[0]

                if (len(ds_fx) > 1) & (len(fs_fx) > 1):
                    ds_dx = np.where(((DSrc.loc[ds_sx[ds_fx], 'expMidptMJD'] <= SNIa_peak_date[sn]-300) 
                                      | (DSrc.loc[ds_sx[ds_fx], 'expMidptMJD'] >= SNIa_peak_date[sn]+500)))[0]
                    fs_dx = np.where(((FSrc.loc[fs_sx[fs_fx], 'expMidptMJD'] <= SNIa_peak_date[sn]-300) 
                                      | (FSrc.loc[fs_sx[fs_fx], 'expMidptMJD'] >= SNIa_peak_date[sn]+500)))[0]

                    if (len(ds_dx) > 1) & (len(fs_dx) > 1):
                        for x in ds_dx:
                            tx = np.where(DSrc.loc[ds_sx[ds_fx[x]], 'ccdVisitId'] == 
                                          FSrc.loc[fs_sx[fs_fx[fs_dx[:]]], 'ccdVisitId'])[0]
                            if len(tx) == 1:
                                ax[i,j].plot(DSrc.loc[ds_sx[ds_fx[x]], 'psFlux'],
                                             FSrc.loc[fs_sx[fs_fx[fs_dx[tx]]], 'psfDiffFlux'],
                                             plot_filter_symbols[filt],
                                             color = plot_filter_colors[filt],
                                             alpha=0.3, label = filt)
                                
                    del ds_dx, fs_dx
                    
                del ds_fx, fs_fx
                # ax[i,j].legend(loc='best')
                f += 1
                
    del ds_sx, fs_sx

ax[2,0].set_xlabel('detected flux')
ax[2,1].set_xlabel('detected flux')
ax[0,0].set_ylabel('forced flux')
ax[1,0].set_ylabel('forced flux')
ax[2,0].set_ylabel('forced flux')
plt.show()

For off-peak detections, at leasted the detection and forced fluxes match.

Still not sure what's going on with the `DiaSource` minimum fluxes indicating a much higher fraction of SNIa with template contamination than `ForcedSourceOnDiaObject` minimum fluxes though.

Next steps might be to just skip the whole part of using the `DiaObject` summary parameters and go straight to using only `ForcedSourceOnDiaObject`. That's the best way to quantify off-peak flux in the difference images.